# Chapter 6: PROTECT (PR) – Safeguards & Implementation

In Chapter 5, we completed the identification phase—mapping our assets, classifying our data, modeling threats, and quantifying risks. We now possess the intelligence required to make informed security decisions. However, intelligence without action leaves us defenseless. The **PROTECT** function of the NIST CSF 2.0 establishes the safeguards and countermeasures that keep adverse events from disrupting our mission.

Protection is where strategy becomes tangible. It is the implementation of access controls that verify identity before granting entry, the encryption that renders stolen data useless, the secure coding practices that eliminate vulnerabilities before deployment, and the resilient architectures that withstand attacks. For developers, this chapter represents the core of your security craftsmanship—the technical controls you implement in code, configuration, and infrastructure.

We will explore Identity and Access Management (IAM) through the lens of Zero Trust, implement data security controls that satisfy regulatory requirements, establish the Secure Software Development Lifecycle (SSDLC) that prevents defects, and harden the platforms upon which our applications run. By the end, you will understand not just *what* controls to implement, but *how* to implement them correctly, efficiently, and verifiably.

---

## 6.1 Identity & Access Management (IAM): MFA, SSO, PAM, and Least Privilege

Identity is the new perimeter. With cloud computing, remote work, and microservices, the traditional network boundary has dissolved. **Identity**—whether human, service account, or machine—has become the primary control point for access to resources.

### 6.1.1 Authentication: Verifying Identity
Authentication answers the question: "Are you who you claim to be?" Strong authentication requires multiple factors:

*   **Something you know:** Passwords, PINs (knowledge factors).
*   **Something you have:** Hardware keys (YubiKey), smartphones (possession factors).
*   **Something you are:** Fingerprints, facial recognition (inherence factors).

**Multi-Factor Authentication (MFA)**
MFA requires two or more factors. It mitigates the risk of credential theft (phishing, password dumps). **FIDO2/WebAuthn** (hardware security keys) provides phishing-resistant MFA, superior to SMS or TOTP (Time-based One-Time Passwords) which can be intercepted or SIM-swapped.

**Implementation: Python MFA Verification Flow**
```python
import pyotp
import qrcode
from cryptography.fernet import Fernet

class MFAController:
    def __init__(self):
        self.fernet = Fernet(os.environ['MFA_SECRET_KEY'])
    
    def enroll_totp(self, user_id):
        """
        Enroll user in TOTP (Time-based One-Time Password)
        Returns: QR code URI for authenticator apps
        """
        # Generate secret (base32 encoded)
        secret = pyotp.random_base32()
        
        # Encrypt secret for storage (never store plaintext!)
        encrypted_secret = self.fernet.encrypt(secret.encode())
        
        # Store in database
        db.store_mfa_secret(user_id, encrypted_secret)
        
        # Generate provisioning URI for QR code
        totp = pyotp.TOTP(secret)
        provisioning_uri = totp.provisioning_uri(
            name=user_id,
            issuer_name="SecureApp Corp"
        )
        
        return provisioning_uri, secret  # Secret shown once for backup
    
    def verify_totp(self, user_id, token):
        """
        Verify 6-digit TOTP code
        """
        encrypted_secret = db.get_mfa_secret(user_id)
        secret = self.fernet.decrypt(encrypted_secret).decode()
        
        totp = pyotp.TOTP(secret)
        
        # Verify with 1-step tolerance for clock skew
        is_valid = totp.verify(token, valid_window=1)
        
        # Prevent replay attacks
        if is_valid and not db.is_token_used(user_id, token):
            db.mark_token_used(user_id, token, window=30)  # 30-second window
            return True
        return False

# WebAuthn (FIDO2) implementation for phishing-resistant MFA
# Uses pyfido2 library for hardware key support
from fido2.webauthn import PublicKeyCredentialCreationOptions
from fido2.server import Fido2Server

class WebAuthnController:
    def __init__(self):
        self.server = Fido2Server({"id": "example.com", "name": "Secure App"})
    
    def begin_registration(self, user_id):
        options, state = self.server.register_begin(
            {
                "id": user_id.encode(),
                "name": user_id,
                "displayName": user_id,
            },
            user_verification="discouraged",  # or "required" for biometric
            authenticator_attachment="platform"  # "cross-platform" for USB keys
        )
        session['fido_state'] = state
        return options
```

### 6.1.2 Authorization: Enforcing Least Privilege
Once authenticated, authorization determines *what* you can do. The **Principle of Least Privilege (PoLP)** dictates that users and processes receive only the minimum permissions necessary to perform their function, for the minimum time necessary.

**Role-Based Access Control (RBAC)**
Permissions are assigned to roles, and users are assigned to roles.
*   **Admin:** Full access
*   **Developer:** Read/write to dev environments, read-only to production logs
*   **Service Account:** API access only to specific endpoints

**Attribute-Based Access Control (ABAC)**
More granular than RBAC. Access decisions based on attributes of the user, resource, and environment.
*   *Example:* "Allow access IF user.department == 'Finance' AND resource.classification == 'Internal' AND time.hour BETWEEN 9 AND 17 AND location.country == 'US'"

**Implementation: ABAC with Open Policy Agent (Rego)**
```rego
# abac_policy.rego
package app.abac

default allow = false

# Allow developers to read non-production data during business hours
allow {
    input.user.role == "developer"
    input.resource.environment != "production"
    input.action == "read"
    time.now_ns >= time.parse_rfc3339_ns("2026-01-01T09:00:00Z")
    time.now_ns <= time.parse_rfc3333_ns("2026-01-01T17:00:00Z")
}

# Allow access to sensitive data only with MFA and from corporate IP
allow {
    input.user.clearance >= input.resource.classification_level
    input.request.mfa_verified == true
    net.cidr_contains("203.0.113.0/24", input.request.remote_ip)
}
```

### 6.1.3 Privileged Access Management (PAM)
Privileged accounts (root, admin, service accounts with broad permissions) are high-value targets. **PAM** controls these accounts through:
*   **Just-in-Time (JIT) Access:** Elevation is temporary (e.g., 2 hours) and requires approval.
*   **Break-Glass Procedures:** Emergency access with full auditing.
*   **Session Recording:** Recording of all privileged sessions for forensic analysis.

**Implementation: Temporary Credential Generation**
```python
import boto3
from datetime import datetime, timedelta

class JITAccessController:
    def request_elevation(self, user_id, role_arn, reason):
        """
        Request temporary privileged access (AWS IAM example)
        """
        # Approval workflow (simplified)
        if not self.approve_request(user_id, reason):
            raise PermissionDenied("Elevation request denied")
        
        sts = boto3.client('sts')
        
        # Assume role with temporary credentials (1-hour duration)
        response = sts.assume_role(
            RoleArn=role_arn,
            RoleSessionName=f"{user_id}-{datetime.now().timestamp()}",
            DurationSeconds=3600,  # 1 hour max
            Tags=[
                {'Key': 'UserId', 'Value': user_id},
                {'Key': 'Reason', 'Value': reason[:256]},
                {'Key': 'ApprovedBy', 'Value': 'security-team'}
            ]
        )
        
        # Log the elevation
        audit_log.info(f"Privilege elevation: {user_id} -> {role_arn} for {reason}")
        
        return {
            'access_key': response['Credentials']['AccessKeyId'],
            'secret_key': response['Credentials']['SecretAccessKey'],
            'session_token': response['Credentials']['SessionToken'],
            'expiration': response['Credentials']['Expiration']
        }
```

### 6.1.4 Zero Trust Architecture
**Zero Trust** eliminates implicit trust based on network location. Every access request is fully authenticated, authorized, and encrypted before access is granted.

**Core Principles:**
1.  **Never Trust, Always Verify:** No "trusted internal network."
2.  **Assume Breach:** Minimize blast radius through micro-segmentation.
3.  **Verify Explicitly:** Use all available signals (identity, device health, location, data classification).

**Implementation: Zero Trust Network Access (ZTNA)**
Instead of VPNs that grant network-level access, ZTNA provides application-level access based on identity.

---

## 6.2 Data Security: Classification, Encryption at Rest/Transit, and DLP

Data is the asset attackers target. Protecting data requires understanding its lifecycle: creation, storage, processing, transmission, and destruction.

### 6.2.1 Enforcing Data Classification
From Chapter 5, we classified data as Public, Internal, Confidential, or Restricted. Protection mechanisms must enforce these labels automatically.

**Implementation: Automated Data Labeling**
```python
from enum import Enum

class DataClassification(Enum):
    PUBLIC = 1
    INTERNAL = 2
    CONFIDENTIAL = 3
    RESTRICTED = 4

class DataProtectionHandler:
    def __init__(self):
        self.encryption_required = {
            DataClassification.PUBLIC: False,
            DataClassification.INTERNAL: False,
            DataClassification.CONFIDENTIAL: True,
            DataClassification.RESTRICTED: True
        }
        
        self.allowed_transports = {
            DataClassification.PUBLIC: ['http', 'https'],
            DataClassification.INTERNAL: ['https'],
            DataClassification.CONFIDENTIAL: ['https', 'sftp'],
            DataClassification.RESTRICTED: ['https-tls13-only']
        }
    
    def process_data(self, data, classification):
        """
        Apply protection controls based on classification
        """
        if self.encryption_required[classification]:
            # Apply AES-256-GCM encryption from Chapter 3
            data = self.encrypt_data(data, key_id=f"class-{classification.name}-key")
        
        # Add metadata headers for downstream protection
        metadata = {
            'classification': classification.name,
            'encryption_status': self.encryption_required[classification],
            'retention_days': self.get_retention_period(classification),
            'masked_fields': self.get_pii_fields(classification)
        }
        
        return data, metadata
```

### 6.2.2 Encryption at Rest and in Transit
**At Rest:** Data stored on disks, databases, or object storage.
*   **Database:** Transparent Data Encryption (TDE) or application-level encryption.
*   **File Storage:** Full Disk Encryption (FDE) with BitLocker (Windows) or LUKS (Linux).
*   **Object Storage:** Server-Side Encryption (SSE-S3, SSE-KMS) or Client-Side Encryption (CSE).

**In Transit:** Data moving between systems.
*   **TLS 1.3:** Mandatory for all communications (see Chapter 3).
*   **Certificate Pinning:** For mobile apps, pin specific certificates to prevent rogue CA attacks.
*   **mTLS (Mutual TLS):** Both client and server present certificates, common in microservices.

**Implementation: Database Field-Level Encryption**
```python
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
import base64

class FieldEncryption:
    """
    Transparent field-level encryption for PII in databases
    """
    def __init__(self, key):
        self.aesgcm = AESGCM(key)
    
    def encrypt_field(self, plaintext: str, associated_data: bytes = None) -> str:
        """
        Encrypts a database field with authentication
        """
        nonce = os.urandom(12)
        ciphertext = self.aesgcm.encrypt(
            nonce, 
            plaintext.encode('utf-8'), 
            associated_data
        )
        # Store as: nonce + ciphertext, base64 encoded
        encrypted_package = base64.b64encode(nonce + ciphertext).decode('utf-8')
        return f"ENC:{encrypted_package}"
    
    def decrypt_field(self, encrypted_value: str, associated_data: bytes = None) -> str:
        """
        Decrypts field if it has ENC: prefix
        """
        if not encrypted_value.startswith("ENC:"):
            return encrypted_value  # Plaintext, migrate to encryption
        
        package = base64.b64decode(encrypted_value[4:])
        nonce = package[:12]
        ciphertext = package[12:]
        
        plaintext = self.aesgcm.decrypt(nonce, ciphertext, associated_data)
        return plaintext.decode('utf-8')

# Usage in ORM (SQLAlchemy example)
from sqlalchemy import TypeDecorator, Text

class EncryptedString(TypeDecorator):
    impl = Text
    
    def process_bind_param(self, value, dialect):
        if value is not None:
            return field_encrypter.encrypt_field(value, b"user_table")
        return value
    
    def process_result_value(self, value, dialect):
        if value is not None:
            return field_encrypter.decrypt_field(value, b"user_table")
        return value

# Model definition
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    email = Column(EncryptedString(255))  # Automatically encrypted
    name = Column(String(100))  # Not sensitive, plaintext
```

### 6.2.3 Data Loss Prevention (DLP)
DLP prevents sensitive data from leaving authorized boundaries via email, uploads, or exfiltration.

**Technical Controls:**
*   **Endpoint DLP:** Prevents copying to USB, blocks screen captures of sensitive apps.
*   **Network DLP:** Inspects traffic for SSNs, credit cards, leaving the network.
*   **Cloud DLP:** CASB (Cloud Access Security Broker) monitors SaaS usage.

**Implementation: Content Inspection Regex (Simplified DLP)**
```python
import re

class DLPScanner:
    def __init__(self):
        self.patterns = {
            'SSN': r'\b\d{3}-\d{2}-\d{4}\b',
            'CREDIT_CARD': r'\b(?:\d[ -]*?){13,16}\b',
            'API_KEY': r'sk_[a-zA-Z0-9]{24,}'
        }
        self.blocked_classifications = ['RESTRICTED']
    
    def scan_content(self, content, metadata):
        """
        Scan outgoing content for sensitive data
        """
        violations = []
        
        for pattern_name, pattern in self.patterns.items():
            if re.search(pattern, content):
                violations.append({
                    'type': pattern_name,
                    'severity': 'HIGH' if metadata['classification'] in self.blocked_classifications else 'MEDIUM'
                })
        
        if violations:
            self.trigger_blocking_action(content, violations)
            raise DataLossPreventionError(f"Sensitive data detected: {violations}")
        
        return True
```

---

## 6.3 Secure Software Development Lifecycle (SSDLC)

The SSDLC integrates security activities into every phase of software development, preventing vulnerabilities rather than patching them later.

### 6.3.1 SSDLC Phases and Security Activities

| Phase | Security Activity | Tools/Methods |
|-------|------------------|---------------|
| **Requirements** | Security requirements, abuse cases | OWASP ASVS, threat modeling |
| **Design** | Architecture risk analysis, secure design patterns | STRIDE, security checklists |
| **Implementation** | Secure coding standards, peer review | SAST, linters, pre-commit hooks |
| **Testing** | Security testing, fuzzing, penetration testing | DAST, IAST, Burp Suite |
| **Deployment** | Secure configuration, hardening | IaC scanning, container scanning |
| **Maintenance** | Patch management, vulnerability monitoring | Dependency checkers, bug bounty |

### 6.3.2 Secure Coding Standards
Enforce standards through automation:

**Pre-commit Hooks (Git)**
```yaml
# .pre-commit-config.yaml
repos:
  - repo: https://github.com/pre-commit/pre-commit-hooks
    rev: v4.5.0
    hooks:
      - id: detect-private-key  # Block private keys
      - id: check-added-large-files  # Block large files (data exfiltration)
  
  - repo: https://github.com/PyCQA/bandit
    rev: 1.7.5
    hooks:
      - id: bandit
        args: ["-ll", "-ii"]  # Low confidence, ignore internal warnings
  
  - repo: https://github.com/trufflesecurity/trufflehog
    rev: v3.63.0
    hooks:
      - id: trufflehog
        args: ["--only-verified", "--fail"]
```

### 6.3.3 Secrets Management
Hardcoded credentials are a critical vulnerability. Use dedicated secrets managers.

**Implementation: HashiCorp Vault Integration**
```python
import hvac
import os

class SecretManager:
    def __init__(self):
        self.client = hvac.Client(
            url=os.environ['VAULT_ADDR'],
            token=os.environ['VAULT_TOKEN']
        )
    
    def get_database_creds(self, role):
        """
        Dynamic database credentials (auto-rotated, short-lived)
        """
        response = self.client.secrets.database.generate_credentials(
            name=role,
            mount_point="database"
        )
        
        return {
            'username': response['data']['username'],
            'password': response['data']['password'],
            'lease_duration': response['lease_duration']
        }
    
    def get_api_key(self, service):
        """
        Static secrets with versioning
        """
        response = self.client.secrets.kv.v2.read_secret_version(
            path=f"api-keys/{service}"
        )
        return response['data']['data']['key']

# Usage - no hardcoded credentials
db_creds = secret_manager.get_database_creds("app-reader-role")
conn = psycopg2.connect(
    host="db.prod.internal",
    user=db_creds['username'],
    password=db_creds['password'],
    # Credentials expire automatically, forcing rotation
)
```

---

## 6.4 Platform Security: Hardening Applications, APIs, and Cloud Infrastructure

### 6.4.1 Application Hardening
*   **Error Handling:** Never expose stack traces or SQL details to users (OWASP A05).
*   **Security Headers:** Implement CSP, HSTS, X-Frame-Options.
*   **Dependency Management:** Keep libraries updated (OWASP A06).

**Implementation: Security Headers Middleware (Python/Flask)**
```python
from flask import Flask
from flask_talisman import Talisman

app = Flask(__name__)

# Force HTTPS and set security headers
Talisman(app, 
         force_https=True,
         strict_transport_security=True,
         content_security_policy={
             'default-src': "'self'",
             'script-src': "'self'",
             'style-src': "'self' 'unsafe-inline'",
             'img-src': "'self' data:",
         },
         feature_policy={
             'geolocation': "'none'",
             'microphone': "'none'",
             'camera': "'none'"
         })

@app.after_request
def set_security_headers(response):
    response.headers['X-Content-Type-Options'] = 'nosniff'
    response.headers['X-Frame-Options'] = 'DENY'
    response.headers['X-XSS-Protection'] = '1; mode=block'
    response.headers['Referrer-Policy'] = 'strict-origin-when-cross-origin'
    response.headers['Permissions-Policy'] = 'accelerometer=(), camera=(), geolocation=(), gyroscope=(), magnetometer=(), microphone=(), payment=(), usb=()'
    return response
```

### 6.4.2 Infrastructure as Code (IaC) Security
Harden infrastructure before deployment using Policy as Code (Chapter 4).

### 6.4.3 API Security
*   **Rate Limiting:** Prevent abuse and DDoS.
*   **Input Validation:** Strict schema validation (JSON Schema, OpenAPI).
*   **Authentication:** OAuth 2.0 + OIDC for modern apps.

**Implementation: API Rate Limiting (Redis-backed)**
```python
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address

limiter = Limiter(
    key_func=get_remote_address,
    storage_uri="redis://redis:6379",
    strategy="moving-window"  # Smooth rate limiting
)

@app.route("/api/login", methods=["POST"])
@limiter.limit("5 per minute")  # Strict limit on authentication
def login():
    # Implementation
    pass

@app.route("/api/data", methods=["GET"])
@limiter.limit("100 per minute")  # Standard API limit
def get_data():
    # Implementation
    pass

# Custom limit for authenticated users vs anonymous
def user_key():
    return current_user.id if current_user.is_authenticated else get_remote_address()

@limiter.request_filter
def exempt_health_checks():
    return request.path == "/health"
```

---

## 6.5 Awareness & Training: Building a Security-First Development Culture

Technical controls fail without human awareness. The human element—social engineering, misconfiguration, insider threats—remains a critical vulnerability.

### 6.5.1 Security Training Requirements
*   **Onboarding:** All developers complete secure coding training before repository access.
*   **Annual Refresh:** Updates on new threats (AI-generated phishing, novel injection techniques).
*   **Role-Based:** Backend engineers learn injection defense; frontend engineers learn XSS/CSRF.

### 6.5.2 Secure Code Review Training
Teach developers to recognize patterns:
*   **The "Dirty Dozen":** OWASP Top 10 + SSRF, XXE, Insecure Deserialization.
*   **Checklist-Based Review:** Systematic examination of authentication, authorization, input handling, output encoding.

### 6.5.3 Simulated Phishing and Social Engineering
Regular testing with controlled phishing campaigns to measure susceptibility and train recognition.

---

### Chapter Summary

In this chapter, we implemented the **PROTECT** function across multiple layers. We established **Identity and Access Management** controls including MFA, Zero Trust architecture, and Just-in-Time privilege elevation, ensuring that only verified identities access specific resources for limited durations. We enforced **Data Security** through classification-aware encryption at rest and in transit, with field-level database encryption and Data Loss Prevention mechanisms preventing unauthorized exfiltration. We institutionalized the **Secure Software Development Lifecycle (SSDLC)**, integrating security requirements, automated SAST/DAST scanning, and secrets management into the development pipeline rather than as afterthoughts. We hardened **Platforms** through security headers, API rate limiting, and Infrastructure as Code validation, creating resilient foundations for our applications.

Protection establishes our defensive posture, but defenses are never perfect. The next layer of defense assumes that despite our best protective controls, adversaries may breach our perimeter, exploit a zero-day vulnerability, or compromise a credential. We must assume breach and build the capability to **DETECT** these intrusions as rapidly as possible, minimizing dwell time and enabling rapid response. We transition from building walls to installing motion sensors and security cameras within our digital estate.

**Next Up: Chapter 7: DETECT (DE) – Continuous Monitoring & Discovery**

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='5. identify_id_asset_risk_management.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='7. detect_de_continuous_monitoring_discovery.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
